In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, GPTNeoModel
import torch
import numpy as np
np.random.seed(20250410)

Let's download a text dataset on which we'll analyze model embeddings. The `fineweb-edu` dataset was gathered by scraping text off webpages and keeping only those that scored high in an "educational contet" metric.

In [2]:
fw = load_dataset("HuggingFaceFW/fineweb-edu", name="CC-MAIN-2024-10", split="train", streaming=True)

Resolving data files:   0%|          | 0/2110 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

To get a sense of what's in the data, let's print a few examples.

In [3]:
n_stream = 20
texts = []
for x in fw:
    texts.append(x["text"])
    if len(texts) > n_stream: break

[f"{s[:200]}..." for s in texts[:10]]

['- It means objects are Garbage Collected more quickly. (Incorrect).\n- Its a good way to make sure all your references are set to null. (Not necessary).\n- Its good practice to implement all the time. (...',
 'CANUSWEST and CANUSWEST North were developed to assist Federal, State/Provincial, local, and Tribal/Aboriginal responders to mitigate the effects of oil and hazardous materials spills on human health ...',
 '– Computer viruses are parasitic programs which are able to replicate themselves, attach themselves to other executables in the computer, and perform some unwanted and often malicious actions. A virus...',
 'For those unfamiliar with Cornish, it is classed as a p-Celtic member of the family of Celtic languages, which was once spoken across much of Europe, and is now restricted to the insular world and Bri...',
 'Democracy is in trouble. No matter what index you look at, the number of countries rated as being fully democratic has declined dramatically over the last twenty ye

The helper function below gets the activations from all the neurons just before the final prediction step. We could have worked with any layer that we want, but this pre-prediction layer should have the highest-level representations. We'll work with a pretrained GPT model, but this conceptually there is nothing tied to this choice.

In [4]:
def extract_embeddings(text, model, tokenizer, layer=-1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=False)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states[layer].mean(axis=(0, 1))


# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = GPTNeoModel.from_pretrained("EleutherAI/gpt-neo-125m")
model = model.eval()

The block below runs that helper function on the first `n_stream` text strings. Each column of `H` corresponds to a neuron, and each row a document.

In [ ]:
from tqdm import tqdm

H = []
for x in tqdm(texts[:n_stream]):
    H.append(extract_embeddings(x, model, tokenizer))
H = torch.stack(H).numpy()


100%|██████████| 20/20 [01:11<00:00,  3.55s/it]


Now, let's define some concepts. The proper, CAV approach would be to train a small classifier to identify samples that represent the topic. We'll use a PCBM-style hack that simply uses text embeddings of strings that represent the concept. In fact, since our data are already text, we don't even need to use any multimodal model. 

In [ ]:
concepts_text = ["literature", "history", "education", "government", "geography", "biology", "computing", "mathematics"]

C = []
for x in tqdm(concepts_text):
    C.append(extract_embeddings(x, model, tokenizer))
C = torch.stack(C).numpy()

100%|██████████| 8/8 [00:01<00:00,  6.14it/s]


The "bottleneck" part of the PCBM trains a simple model using coordinates of the original samples with respect to the concept basis. The block below shows how these can be extracted. To understand the equation, remember that the projection of a vector $h$ onto the columns of a matrix $X$ is given by $X \left(X^\top X\right)^{-1} X^{\top}h$. In our case, we want to project onto the rows of $C$ extracted above, so we should consider $C^{\top}\left(C C^\top\right)^{-1} C h$ instead. Further, rather than the projected value, we want the coordinates of the projection with respect to the basis, so we remove the first $C^\top$. Finally, to compute this across many vectors $h$ simultaneously, we can horizontally concatenate them into the matrix $H^\top$, as done below.

In [7]:
import pandas as pd
pd.set_option("display.max_colwidth", 1000)

coordinates = np.linalg.inv(C @ C.T) @ C @ H.T
coordinates_df = pd.DataFrame(np.round(coordinates.T, 2), columns=concepts_text)
coordinates_df["text"] = texts[:n_stream]

coordinates_df

,literature,history,education,government,geography,biology,computing,mathematics,text
0,0.29,-0.08,0.18,0.06,-0.08,-0.10,0.48,-0.28,"- It means objects are Garbage Collected more quickly. (Incorrect).\n- Its a good way to make sure all your references are set to null. (Not necessary).\n- Its good practice to implement all the time. (Incorrect).\n- The Garbage Collector calls IDisposable.Dispose() automatically. (Incorrect).\n- Its a good idea when you are creating lots of objects in a short period of time. (There is a better way).\nWhat is it?\nThe IDisposable interface seems to be a commonly misunderstood and misused interface. Some developers like to implement it as a matter of course thinking it is good practice. But it is absolutely not necessary when dealing with managed objects (native .Net objects). In .Net the Garbage Collector (GC) is more than capable of finding and disposing all objects quickly and efficiently.\nTo quote Andrew Troelsen: ""Allocate an object onto the heap using the new keyword and forget about it"".\nThe GC stores lists of objects in generations ""young"" objects have a lower generation n..."
1,-0.39,-0.26,0.12,0.29,0.46,0.08,0.01,-0.05,"CANUSWEST and CANUSWEST North were developed to assist Federal, State/Provincial, local, and Tribal/Aboriginal responders to mitigate the effects of oil and hazardous materials spills on human health and safety, environment, and property by specifying the processes needed to facilitate an effective response to environmental emergency incidents on either side of the British Columbia, Canada/USA border. This plan was developed pursuant to the July of 1994 the Canada-United States Joint Inland Pollution Contingency Plan (the Inland Plan) signed by the Administrator of EPA and the Minister for Department of the Environment which divided the common border between the countries into five regions. EPA Region 10 and Environment Canada Pacific and Yukon Regions developed the CANUSWEST and CANUSWEST NORTH Annexes to the Inland Plan to address issues unique to the area. CANUSWEST and CANUSWEST North are based on the principle of escalation and accordingly recognizes the roles of the local, st..."
2,0.26,-0.29,0.13,0.19,-0.11,-0.04,0.60,-0.22,"– Computer viruses are parasitic programs which are able to replicate themselves, attach themselves to other executables in the computer, and perform some unwanted and often malicious actions. A virus is not able to spread itself to another computers, some user actions are needed for it to infect a new computer. Downloading and running software from untrusted sources, inserting an USB drive without a previous scan–remember always disable the AutoRun feature for the drives as CD-ROMs, DVD-ROMs– , downloading and running emails or IM attachments even from known persons, can put you in the nasty situation to have an infected computer. Always when you deal with these situations and to prevent computer infections, scan before to run.\nThe best scanners in my opinion are multi-engine online scanners like virustotal.com or novirusthanks.org. The links of these scanners and many more are on the home page.\nThe main three features of a virus are :\n– the replication mechanism search and fin..."
3,0.35,-0.21,0.06,-0.05,0.30,-0.06,-0.02,-0.05,"For those unfamiliar with Cornish, it is classed as a p-Celtic member of the family of Celtic languages, which was once spoken across much of Europe, and is now restricted to the insular world and Brittany: the only surviving languages being Cornish, Welsh and Breton (all p-Celtic), and Manx, Scots Gaelic and Irish (all q-Celtic).\nThe relationship between these two branches is illustrated by p-Celtic words such as peduar W and their q-Celtic equivalents: cethar [Ir].\nThe etymology, morphology, syntax and phonology of Cornish and the other Celtic languages ultimately derive from a putative proto-Indo European or proto-Celtic language or family of languages spoken in Britain in pre-history.\nCornish Onomastics